In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, LatentDirichletAllocation
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
import warnings
import random
warnings.filterwarnings("ignore")

# Gera seed aleatório
seed = random.randint(1, 1_000)
print(seed)

698


# Seção - Gerar dados sintéticos


In [3]:
rng = np.random.default_rng(seed)

N_CUSTOMERS = 300
N_ORDERS = 1200
CITIES = ["Rio de Janeiro", "São Paulo", "Belo Horizonte", "Curitiba", "Porto Alegre", "Salvador", "Recife"]
CHANNELS = ["website", "app", "loja_fisica", "telefone"]
CATEGORIES = ["eletronicos", "livros", "roupas", "alimentos", "casa", "esporte"]
START_DATE = datetime(2024, 1, 1)

# Base de clientes
customers = pd.DataFrame({
    "customer_id": np.arange(1, N_CUSTOMERS + 1),
    "idade": rng.integers(18, 75, size=N_CUSTOMERS),
    "cidade": rng.choice(CITIES, size=N_CUSTOMERS, replace=True),
    "renda_mensal": rng.normal(5000, 2000, size=N_CUSTOMERS).clip(1500, 20000),
    "data_cadastro": [START_DATE + timedelta(days=int(x)) for x in rng.integers(0, 400, size=N_CUSTOMERS)],
})

# Base de pedidos
orders = pd.DataFrame({
    "order_id": np.arange(1, N_ORDERS + 1),
    "customer_id": rng.integers(1, N_CUSTOMERS + 1, size=N_ORDERS),
    "categoria": rng.choice(CATEGORIES, size=N_ORDERS, replace=True),
    "canal": rng.choice(CHANNELS, size=N_ORDERS, replace=True),
    "quantidade": rng.integers(1, 6, size=N_ORDERS),
    "preco_unitario": (rng.normal(150, 80, size=N_ORDERS).clip(10, 1200)).round(2),
    "devolvido": rng.choice([0, 1], size=N_ORDERS, p=[0.9, 0.1])
})
orders["valor"] = (orders["quantidade"] * orders["preco_unitario"]).round(2)
orders["data_pedido"] = [START_DATE + timedelta(days=int(x)) for x in rng.integers(0, 400, size=N_ORDERS)]

# Pequena base de produtos para merge posterior
products = pd.DataFrame({
    "categoria": CATEGORIES,
    "margem_padrao": [0.25, 0.30, 0.35, 0.20, 0.22, 0.28],
    "departamento": ["tech", "midia", "vestuario", "alimentos", "casa", "esportes"]
})

# Introduzir alguns NaNs artificialmente
mask_nan = rng.choice([True, False], size=len(orders), p=[0.05, 0.95])
orders.loc[mask_nan, "preco_unitario"] = np.nan

In [4]:
customers

,customer_id,idade,cidade,renda_mensal,data_cadastro
0,1,65,São Paulo,3913.380990,2024-02-20
1,2,55,Curitiba,5546.187356,2024-06-13
2,3,23,Curitiba,7874.611192,2024-12-23
3,4,69,Salvador,5725.404662,2024-08-27
4,5,43,Recife,7252.884144,2024-09-07
...,...,...,...,...,...
295,296,51,Belo Horizonte,3979.067430,2024-04-21
296,297,50,Recife,3815.979848,2024-09-13
297,298,37,Recife,2397.759170,2024-08-16
298,299,42,Recife,4026.121709,2024-01-20


In [5]:
orders

,order_id,customer_id,categoria,canal,quantidade,preco_unitario,devolvido,valor,data_pedido
0,1,106,alimentos,telefone,5,15.20,0,76.00,2024-04-13
1,2,40,casa,telefone,1,126.46,1,126.46,2024-12-11
2,3,187,esporte,telefone,4,152.41,0,609.64,2024-07-30
3,4,14,alimentos,website,3,10.00,0,30.00,2024-05-03
4,5,156,eletronicos,app,3,271.95,1,815.85,2024-07-12
...,...,...,...,...,...,...,...,...,...
1195,1196,109,livros,website,3,10.00,0,30.00,2024-01-11
1196,1197,46,alimentos,app,4,147.95,0,591.80,2024-04-20
1197,1198,40,roupas,app,1,218.89,0,218.89,2024-03-16
1198,1199,209,eletronicos,website,1,63.50,0,63.50,2024-01-03


# Exercício 1

Filtre todos os pedidos com valores maior que 500, feitos pelo website, e que não foram devolvidos. Retorne apenas as colunas order_id, customer_id, valor, canal, devolvido.

# Exercício 2

Agrupe a base de pedidos por categoria e calcule:

- total de pedidos (contagem)
- valor total (soma de valor)
- ticket médio (média de valor)
- taxa de devolução (média de devolvido)

Ordene pelo valor total decrescente (método `.sort_values()`)

# Exercício 3
Renomeie as colunas de customers para o padrão snake_case em inglês:

(idade -> age, cidade -> city, renda_mensal -> monthly_income, data_cadastro -> signup_date).

Mantenha 'customer_id' igual. Crie um novo DataFrame chamado customers_renamed.

> Snake case (ou snake_case) é uma convenção de nomenclatura na programação em que as palavras são separadas por um traço baixo (_), e todas as letras são minúsculas. Por exemplo, primeiro_nome ou tab_news são exemplos de snake case

# Exercício 4

Verifique a quantidade de valores ausentes por coluna em 'orders'.

Depois, crie uma versão 'orders_filled' onde 'preco_unitario' ausente é preenchido pela mediana da categoria correspondente.

O método que utilizamos para substituir os NAs é `.fillna()`. Leia a documentação para entender como utilizá-la.


# Exercício 5

Converta 'data_pedido' para datetime (se necessário) e gere uma série mensal com número de pedidos e valor total no mês. Junte as duas séries em um único DataFrame mensal.

> A função que converse para datetime é `pd.to_datetime()`. Use o método .`.resample()` para entender como reordenar por mês. Leia a documentação.

# Exercício 6

Faça um merge (left) entre 'orders_filled' e 'products' pela coluna 'categoria'.

Crie a coluna 'lucro_estimado' = valor * margem_padrao. Mostre as 5 primeiras linhas.

> Procure na documentação sobre o método `.merge()`.

# Exercício 7

Crie uma tabela dinâmica (método `.pivot_table()`) com índice 'categoria', colunas 'canal' e valores 'valor' (soma).

Preencha ausências com 0 e adicione uma coluna 'total' (soma por linha). Ordene por total desc.

# Exercício 8

Para cada cliente, calcule métricas RFM simples:

> R (recência em dias desde o último pedido), F (nº de pedidos), M (valor total).

Em seguida, padronize (StandardScaler), reduza a 2 componentes com PCA e faça KMeans(k=3).

Retorne o DataFrame de clientes com cluster e as componentes do PCA.

# Exercício 9

Crie uma coluna 'segmento_valor' em orders_filled com regra:

- 'alto' se valor >= p80 da distribuição de 'valor'
- 'medio' se entre p50 (inclusive) e p80 (exclusive)
- 'baixo' se abaixo de p50

Em seguida, calcule a distribuição (%) de 'segmento_valor' por canal (linha soma 100%).